In [1]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings
import numpy as np
warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('../train/*.csv'))]
from matplotlib import pyplot as plt

In [2]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [10]:
from sklearn.preprocessing import LabelEncoder
years = ['2017', '2018', '2019', '2020']

year_encoder = {}

for year, df in zip(years, know_train):
    print(year)
    encoders = {}
    
    for col in df.columns:
        if col == 'ID':
            continue
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = LabelEncoder()
            df[col] = df[col].map(str)
            df[col] = encoder.fit_transform(df[col])
            encoders[col] = encoder
            
            
    year_encoder[year] = encoders

2017
2018
2019
2020


In [13]:
data = {}
for year, df in zip(years, know_train):
    data[year] = df.drop('idx',axis=1)

In [25]:
data['2017']['bq36'].value_counts()

1    6506
2    2980
Name: bq36, dtype: int64

In [23]:
data['2017'].loc[data['2017']['aq4_1']==1]['aq4_2'].value_counts()

0    3118
Name: aq4_2, dtype: int64

In [2]:
# from sklearn.preprocessing import OneHotEncoder
# onehot_encoder=OneHotEncoder()
# one_col = {'2017' : 'aq41_2',
#           '2018' : 'iq6',
#           '2019' : 'kq33_2',
#           '2020' : 'vq13'}

# one_df = {}
# for key,val in one_col.items():
#     for i in range(data[key].loc[:,:val].shape[1]):
#         if i == 0 :
#             test_df = pd.DataFrame(onehot_encoder.fit_transform(data[key].iloc[:,i].values.reshape(-1,1)).toarray())
#         else:
#             cat = pd.DataFrame(onehot_encoder.fit_transform(data[key].iloc[:,i].values.reshape(-1,1)).toarray())
#             test_df = pd.concat([test_df,cat],axis=1)
            
#         test_df = test_df.fillna(0)
        
#     one_df[key] = pd.concat([test_df,data[key].loc[:,val:]],axis=1)

In [6]:
train_data = {}
for year in years:
    train_data[year] = {'X': one_df[year].iloc[:, 1:-1], # ID제외
                        'y': one_df[year].iloc[:, -1]} 

In [7]:
from sklearn.preprocessing import StandardScaler
minmax = StandardScaler()
for year in years:
    train_data[year]['X'] = minmax.fit_transform(train_data[year]['X'])
    train_data[year]['X'] = pd.DataFrame(train_data[year]['X'])

In [8]:
from sklearn.model_selection import train_test_split

split_data = {}
for year in tqdm(years):
    X_train,X_test,y_train,y_test = train_test_split(train_data[year]['X'],train_data[year]['y'],test_size=0.25, random_state=42)
    split_data[year] = {'X_train' : X_train,
                       'y_train' : y_train,
                       'X_test' : X_test,
                       'y_test' : y_test}

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 22.33it/s]


In [9]:
del train_data

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf_models = {}

for year in tqdm(years):
    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(split_data[year]['X_train'], split_data[year]['y_train'])
    rf_models[year] = model
    

 25%|█████████████████████                                                               | 1/4 [00:09<00:29,  9.85s/it]


KeyboardInterrupt: 

In [11]:
from sklearn.metrics import f1_score
rf_f1 =[]
for year in tqdm(years):
    y_pred = rf_models[year].predict(split_data[year]['X_test'])
    rf_f1.append(f1_score(split_data[year]['y_test'],y_pred,average ='micro'))
np.mean(rf_f1)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.22it/s]


0.4494286814736278

In [ ]:

from imblearn.over_sampling import SMOTE

sm = SMOTE(ratio='auto', kind='regular')

X_resampled, y_resampled = sm.fit_sample(X_train,list(y_train))